# Implement secure Azure solutions: interactive walkthrough

This notebook complements `notes.md` by demonstrating Azure CLI sequences and Python SDK patterns for Key Vault, managed identities, and App Configuration. Adjust the configuration variables in the code cells so the commands or snippets reflect your own environment before running them.

## Optional setup

Run the next cell once to install the Azure SDK packages used in the Python examples. If the packages are already present in your environment you can skip this step.

In [ ]:
%pip install --quiet azure-identity azure-keyvault-secrets azure-storage-blob azure-messaging-eventhubs azure-appconfiguration

In [ ]:
# Import shared dependencies and expose helper utilities.
#
# The utilities keep the notebook runnable even when the Azure SDK is not installed
# or when you do not have access to live Azure resources.
import os
import secrets
from textwrap import dedent
from typing import Optional

try:
    from azure.identity import (
        AzureCliCredential,
        ChainedTokenCredential,
        CredentialUnavailableError,
        DefaultAzureCredential,
        ManagedIdentityCredential,
    )
    from azure.core.exceptions import ClientAuthenticationError
    from azure.keyvault.secrets import SecretClient
    from azure.messaging.eventhubs import EventHubProducerClient
    from azure.storage.blob import BlobClient
    azure_sdks_available = True
except ImportError as import_exc:
    AzureCliCredential = None
    ChainedTokenCredential = None
    CredentialUnavailableError = None
    DefaultAzureCredential = None
    ManagedIdentityCredential = None
    ClientAuthenticationError = None
    SecretClient = None
    EventHubProducerClient = None
    BlobClient = None
    azure_sdks_available = False
    missing_package_error = import_exc
else:
    missing_package_error = None

if azure_sdks_available:
    credential_error_types = (CredentialUnavailableError, ClientAuthenticationError)
else:
    credential_error_types = (Exception,)

def report_sdk_status() -> None:
    if azure_sdks_available:
        print("Azure SDK packages detected. Live credentials will be used when possible.")
    else:
        print("Azure SDK packages not detected. Install them via %pip to run live calls.")
        print(f"Falling back to simulated responses: {missing_package_error}")

report_sdk_status()

## Azure Key Vault: provisioning flow


Generate a Key Vault name and print the CLI commands used to create the resource group, provision the vault, add a secret, retrieve it, and optionally clean up. Nothing is executed automatically; the commands are printed so you can copy/paste them into a terminal that has the Azure CLI authenticated.


In [ ]:
def key_vault_cli_script(

    location: str,

    resource_group: str,

    secret_name: str = "ExamplePassword",

    secret_value: str = "hVFkk965BuUv",

):

    generated_vault_name = os.environ.get("AZURE_KEY_VAULT_NAME") or f"az204vault-{secrets.randbits(24):06x}"

    script = dedent(

        f"""

        # Set reusable variables

        myKeyVault={generated_vault_name}

        myLocation={location}



        # Create a resource group

        az group create --name {resource_group} --location $myLocation



        # Provision the Key Vault

        az keyvault create --name $myKeyVault --resource-group {resource_group} --location $myLocation



        # Store a secret

        az keyvault secret set --vault-name $myKeyVault --name "{secret_name}" --value "{secret_value}"



        # Retrieve the secret

        az keyvault secret show --name "{secret_name}" --vault-name $myKeyVault



        # Optional cleanup

        az group delete --name {resource_group} --no-wait

        """

    ).strip()

    return generated_vault_name, script



location = os.environ.get("AZURE_LOCATION", "eastus")

resource_group = os.environ.get("AZURE_RESOURCE_GROUP", "az204-vault-rg")

vault_name, cli_script = key_vault_cli_script(location=location, resource_group=resource_group)



print(f"Generated vault name: {vault_name}")

print("\nRun these commands in the Azure CLI once you're ready:\n")

print(cli_script)


## Azure Key Vault: Python SDK walkthrough

The following helper demonstrates how `DefaultAzureCredential` authenticates and how the `SecretClient` retrieves a secret. If the required SDK packages or live credentials are missing, the helper falls back to a simulated response so you can still observe the control flow.


In [ ]:
def retrieve_secret(vault_url: str, secret_name: str, simulate: Optional[bool] = None):

    if simulate is None:

        simulate = (not azure_sdks_available) or ("<your-key-vault-name>" in vault_url)

    if simulate:

        print("Simulation mode: constructing the SecretClient flow without calling Azure.")

        return {"name": secret_name, "value": "*** redacted ***"}



    credential = DefaultAzureCredential(exclude_interactive_browser_credential=False)

    client = SecretClient(vault_url=vault_url, credential=credential)

    try:

        secret = client.get_secret(secret_name)

        return {"name": secret.name, "value": secret.value}

    except credential_error_types as exc:

        print(f"Could not retrieve the secret ({exc}). Falling back to simulated output.")

        return {"name": secret_name, "value": "*** redacted ***"}



demo_vault_url = os.environ.get("AZURE_KEY_VAULT_URL", "https://<your-key-vault-name>.vault.azure.net/")

demo_secret_name = os.environ.get("AZURE_SECRET_NAME", "ExamplePassword")

secret_snapshot = retrieve_secret(demo_vault_url, demo_secret_name)

print("Secret snapshot:", secret_snapshot)


## Managed identities: enablement scripts

The following snippets generate Azure CLI commands for both system-assigned and user-assigned managed identities on virtual machines. Adjust the placeholders as needed.


In [ ]:
def system_assigned_identity_cli(vm_name: str, resource_group: str, role: str, scope: str) -> str:

    return dedent(

        f"""

        az vm create \\

            --resource-group {resource_group} \\

            --name {vm_name} \\

            --image win2016datacenter \\

            --generate-ssh-keys \\

            --assign-identity \\

            --role {role} \\

            --scope {scope} \\

            --admin-username azureuser \\

            --admin-password <replace-with-strong-password>

        """

    ).strip()



def user_assigned_identity_cli(vm_name: str, resource_group: str, identity_name: str, role: str, scope: str) -> str:

    identity_commands = dedent(

        f"""

        # Create the user-assigned managed identity

        az identity create --resource-group {resource_group} --name {identity_name}

        """

    ).strip()

    vm_commands = dedent(

        f"""

        # Attach the identity during VM creation

        az vm create \\

            --resource-group {resource_group} \\

            --name {vm_name} \\

            --image Ubuntu2204 \\

            --admin-username azureuser \\

            --admin-password <replace-with-strong-password> \\

            --assign-identity {identity_name} \\

            --role {role} \\

            --scope {scope}



        # Attach the identity to an existing VM (if needed)

        az vm identity assign \\

            --resource-group {resource_group} \\

            --name {vm_name} \\

            --identities {identity_name}

        """

    ).strip()

    return identity_commands + "\n\n" + vm_commands



vm_name = os.environ.get("AZURE_VM_NAME", "myVM")

identity_rg = os.environ.get("AZURE_VM_RESOURCE_GROUP", "myResourceGroup")

subscription_scope = os.environ.get("AZURE_SUBSCRIPTION_SCOPE", "/subscriptions/<subscription-id>")



print("System-assigned managed identity commands:")

print(system_assigned_identity_cli(vm_name, identity_rg, role="contributor", scope=subscription_scope))



print("\nUser-assigned managed identity commands:")

print(user_assigned_identity_cli(vm_name, identity_rg, identity_name="myUserAssignedIdentity", role="contributor", scope=subscription_scope))


## Managed identities: Python SDK usage

Use a user-assigned managed identity by passing its client ID to `DefaultAzureCredential`. The helper below simulates an upload to Blob Storage while displaying the key steps.


In [ ]:
def upload_with_user_identity(blob_url: str, client_id: str, data: bytes, simulate: Optional[bool] = None):

    if simulate is None:

        simulate = (not azure_sdks_available) or (client_id.startswith("<")) or ("<storage-account>" in blob_url)



    if simulate:

        print("Simulation mode: showing the upload flow without contacting Azure Storage.")

        print(f"Blob URL: {blob_url}")

        print(f"Managed identity client ID: {client_id}")

        print(f"Payload length: {len(data)} bytes")

        return {"blob_url": blob_url, "bytes": len(data)}



    credential = DefaultAzureCredential(managed_identity_client_id=client_id, exclude_interactive_browser_credential=False)

    blob_client = BlobClient.from_blob_url(blob_url=blob_url, credential=credential)

    blob_client.upload_blob(data, overwrite=True)

    print("Upload succeeded.")

    return {"blob_url": blob_url, "bytes": len(data)}



sample_blob_url = os.environ.get("AZURE_BLOB_URL", "https://<storage-account>.blob.core.windows.net/mycontainer/demo.txt")

user_identity_client_id = os.environ.get("AZURE_USER_MANAGED_IDENTITY_CLIENT_ID", "<managed-identity-client-id>")

upload_summary = upload_with_user_identity(sample_blob_url, user_identity_client_id, b"Hello from managed identity!")

print("Upload summary:", upload_summary)


## Custom authentication chains

`ChainedTokenCredential` lets you define the exact fallback order when obtaining tokens. The next helper builds a chain that first attempts managed identity, then falls back to a local Azure CLI session.


In [ ]:
def build_eventhub_client(namespace: str, hub_name: str, simulate: Optional[bool] = None):

    if simulate is None:

        simulate = (not azure_sdks_available) or (namespace.startswith("<"))



    if simulate:

        print("Simulation mode: describing a ChainedTokenCredential flow.")

        print("1. Try managed identity token from the host.")

        print("2. If unavailable, fall back to Azure CLI cached credentials.")

        print(f"Target Event Hub: {namespace} / {hub_name}")

        return {"namespace": namespace, "eventhub": hub_name}



    credential = ChainedTokenCredential(ManagedIdentityCredential(), AzureCliCredential())

    client = EventHubProducerClient(fully_qualified_namespace=namespace, eventhub_name=hub_name, credential=credential)

    print("EventHubProducerClient ready. Use send_batch to publish events.")

    return {"namespace": namespace, "eventhub": hub_name}



eventhub_namespace = os.environ.get("AZURE_EVENTHUB_NAMESPACE", "<event-hub-namespace>.eventhubs.windows.net")

eventhub_name = os.environ.get("AZURE_EVENTHUB_NAME", "myhubpath")

eventhub_summary = build_eventhub_client(eventhub_namespace, eventhub_name)

print("Event Hub client summary:", eventhub_summary)


## Azure App Configuration: feature flags

Feature flags can be represented as JSON data. The next cell evaluates a simple configuration that includes boolean flags and a percentage filter.


In [ ]:
feature_config = {

    "FeatureManagement": {

        "FeatureA": True,

        "FeatureB": False,

        "FeatureC": {

            "EnabledFor": [

                {

                    "Name": "Percentage",

                    "Parameters": {

                        "Value": 50

                    }

                }

            ]

        }

    }

}



def evaluate_feature(config: dict, feature: str, context: Optional[dict] = None) -> bool:

    context = context or {}

    flags = config.get("FeatureManagement", {})

    entry = flags.get(feature)

    if entry is None:

        raise KeyError(f"Unknown feature flag: {feature}")

    if isinstance(entry, bool):

        return entry

    for feature_filter in entry.get("EnabledFor", []):

        name = feature_filter.get("Name")

        if name == "Percentage":

            percentage = feature_filter.get("Parameters", {}).get("Value", 0)

            user_id = context.get("user_id", "guest")

            bucket = sum(ord(char) for char in user_id) % 100

            return bucket < int(percentage)

    return False



print("FeatureA enabled?", evaluate_feature(feature_config, "FeatureA"))

print("FeatureB enabled?", evaluate_feature(feature_config, "FeatureB"))

print("FeatureC enabled for user alice?", evaluate_feature(feature_config, "FeatureC", {"user_id": "alice"}))

print("FeatureC enabled for user bob?", evaluate_feature(feature_config, "FeatureC", {"user_id": "bob"}))


## Azure App Configuration: securing the store

Use managed identities to let App Configuration read customer-managed keys or access private endpoints. The following generator prints CLI commands to assign identities.


In [ ]:
def app_config_identity_commands(store_name: str, resource_group: str, user_identity_id: str) -> str:

    return dedent(

        f"""

        # Assign a system-assigned managed identity

        az appconfig identity assign --name {store_name} --resource-group {resource_group}



        # Assign a user-assigned managed identity

        az appconfig identity assign \\

            --name {store_name} \\

            --resource-group {resource_group} \\

            --identities {user_identity_id}

        """

    ).strip()



config_store = os.environ.get("AZURE_APP_CONFIG_NAME", "myTestAppConfigStore")

config_rg = os.environ.get("AZURE_APP_CONFIG_RG", "myResourceGroup")

user_identity_resource_id = os.environ.get(

    "AZURE_USER_MANAGED_IDENTITY_RESOURCE_ID",

    "/subscriptions/<subscription-id>/resourceGroups/myResourceGroup/providers/Microsoft.ManagedIdentity/userAssignedIdentities/myUserAssignedIdentity",

)

print(app_config_identity_commands(config_store, config_rg, user_identity_resource_id))


## Wrap-up

You now have reference snippets that align with the module: CLI flows for provisioning and securing resources, Python SDK patterns for `DefaultAzureCredential`, managed identities, and App Configuration feature flags. Update the placeholder values with real resource names when you're ready to execute any of the commands against Azure.
